In [2]:
!nvidia-smi
!pip -q install -U uv fastapi "uvicorn[standard]" opencv-python pyngrok

%cd /content
!rm -rf openpi
!git clone --recurse-submodules https://github.com/Physical-Intelligence/openpi.git
%cd /content/openpi

!GIT_LFS_SKIP_SMUDGE=1 uv sync
!GIT_LFS_SKIP_SMUDGE=1 uv pip install -e .

# Make sure pip exists in the venv, then install server deps INTO the venv
!.venv/bin/python -m ensurepip --upgrade
!.venv/bin/python -m pip install -U pip setuptools wheel
!.venv/bin/python -m pip install fastapi "uvicorn[standard]" opencv-python

/bin/bash: line 1: nvidia-smi: command not found
/content
Cloning into 'openpi'...
remote: Enumerating objects: 1561, done.
remote: Total 1561 (delta 0), reused 0 (delta 0), pack-reused 1561 (from 1)
Receiving objects: 100% (1561/1561), 24.64 MiB | 47.60 MiB/s, done.
Resolving deltas: 100% (825/825), done.
Submodule 'third_party/aloha' (https://github.com/Physical-Intelligence/aloha.git) registered for path 'third_party/aloha'
Submodule 'third_party/libero' (https://github.com/Lifelong-Robot-Learning/LIBERO.git) registered for path 'third_party/libero'
Cloning into '/content/openpi/third_party/aloha'...
remote: Enumerating objects: 259, done.        
remote: Counting objects: 100% (91/91), done.        
remote: Compressing objects: 100% (67/67), done.        
remote: Total 259 (delta 43), reused 55 (delta 20), pack-reused 168 (from 1)        
Receiving objects: 100% (259/259), 42.74 MiB | 41.21 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Cloning into '/content/openpi/third_par

In [33]:
%%writefile server_pi05_libero.py
import os
os.environ["GCSFS_TOKEN"] = "anon"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

import argparse
import base64
import traceback

import cv2
import numpy as np
import uvicorn
from fastapi import FastAPI
from fastapi.responses import JSONResponse

from openpi.training import config as _config
from openpi.policies import policy_config
from openpi.shared import download


def decode_b64_jpg_to_rgb(image_b64: str | None):
    if not image_b64:
        return None
    raw = base64.b64decode(image_b64)
    arr = np.frombuffer(raw, dtype=np.uint8)
    bgr = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    if bgr is None:
        return None
    return bgr[..., ::-1]  # RGB


def dummy_rgb(h=224, w=224):
    return np.zeros((h, w, 3), dtype=np.uint8)


cfg = _config.get_config("pi05_libero")

ckpt = download.maybe_download(
    "gs://openpi-assets/checkpoints/pi05_libero/",  # trailing slash
    gs={"token": "anon"},
)

policy = policy_config.create_trained_policy(cfg, ckpt)

app = FastAPI(title="pi05_libero_remote_server")


@app.get("/health")
def health():
    return {"status": "ok"}


@app.post("/infer")
def infer(payload: dict):
    try:
        # Images (fallback to dummy if None)
        base_rgb = decode_b64_jpg_to_rgb(payload.get("observation/exterior_image_1_left_b64_jpg"))
        wrist_rgb = decode_b64_jpg_to_rgb(payload.get("observation/wrist_image_left_b64_jpg"))
        if base_rgb is None:
            base_rgb = dummy_rgb()
        if wrist_rgb is None:
            wrist_rgb = dummy_rgb()

        # State (force correct shapes)
        joint = np.asarray(payload.get("observation/joint_position", []), dtype=np.float32).reshape(-1)
        grip = np.asarray(payload.get("observation/gripper_position", []), dtype=np.float32).reshape(-1)

        if joint.size != 7:
            joint = np.zeros((7,), dtype=np.float32)
        if grip.size != 1:
            grip = np.zeros((1,), dtype=np.float32)

        state = np.concatenate([joint, grip], axis=0)  # len=8

        example = {
            "observation/image": base_rgb,
            "observation/wrist_image": wrist_rgb,
            "observation/state": state,
            "prompt": payload.get("prompt", "pick and place"),
        }

        out = policy.infer(example)
        actions7 = np.asarray(out["actions"], dtype=np.float32)

        # Temporary compatibility with local client expecting [H,8]
        actions8 = np.concatenate(
            [actions7, np.zeros((actions7.shape[0], 1), dtype=np.float32)],
            axis=1,
        )

        return {"actions": actions8.tolist(), "actions7": actions7.tolist()}

    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={
                "error": str(e),
                "traceback": traceback.format_exc()[:4000],
            },
        )


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--host", type=str, default="0.0.0.0")
    parser.add_argument("--port", type=int, default=8000)
    args = parser.parse_args()

    uvicorn.run(app, host=args.host, port=args.port)

Overwriting server_pi05_libero.py


In [133]:
%cd /content/openpi
!nohup .venv/bin/python server_pi05_libero.py --host 0.0.0.0 --port 8000 > server.log 2>&1 &

/content/openpi


In [147]:
# to check server is on
!tail -n 20 /content/openpi/server.log

INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1" 200 OK
INFO:     2605:59ca:6853:b008:28f8:3bdb:8a04:ce6e:0 - "POST /infer HTTP/1.1"

In [144]:
!curl -s http://127.0.0.1:8000/health

{"status":"ok"}

In [145]:
from pyngrok import ngrok

from pyngrok import ngrok

ngrok.set_auth_token("3A99o8GspMC5q7r6e4kOnnN85kO_25Gnk8hTcn5fr1XniArCX")

tunnel = ngrok.connect(8000, "http")
print("Public URL:", tunnel.public_url)


Public URL: https://unmetalised-jolanda-perthitic.ngrok-free.dev


In [146]:
#to kill server:
!pkill -f server_pi05_libero.py

In [124]:
#sample refer
import requests

url = "https://unmetalised-jolanda-perthitic.ngrok-free.dev/infer"
payload = {
    "prompt": "pick and place the large teddy bear in box",
    "observation/joint_position": [0,0,0,0,0,0,0],
    "observation/gripper_position": [0],
    "observation/exterior_image_1_left_b64_jpg": None,
    "observation/wrist_image_left_b64_jpg": None,
}

r = requests.post(url, json=payload, timeout=180)
print("Status:", r.status_code)
print(r.text[:2000])

Status: 502
<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/ibm-plex-mono/IBMPlexMono-Text.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link re